In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

# CSVファイルを読み込む
df = pd.read_csv('./data/access.csv')

# 同一ユーザーの連続したリクエストを取得する
df['next_request'] = df.groupby('IP')['Request'].shift(-1)

# NaNを削除する
df = df.dropna(subset=['next_request'])

# 有向グラフを作成する
G = nx.from_pandas_edgelist(df, 'Request', 'next_request', create_using=nx.DiGraph())

# グラフを描画する
nx.draw(G, with_labels=True)
plt.show()